In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_active
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH assortment_class AS (
    SELECT  
        store_id,
        product_id,
        assortment_class,
        assortment_class_desc   
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_scd
    WHERE status = 'ACTIVE' 
      AND end_date >= DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
      AND assortment_class IS NOT NULL
    QUALIFY ROW_NUMBER() OVER (PARTITION BY store_id, product_id ORDER BY assortment_class ASC) = 1
)
SELECT 
    a.store_id, 
    a.product_id,
    a1.assortment_class,
    a1.assortment_class_desc
FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_current a
LEFT JOIN assortment_class a1 
  ON a.store_id = a1.store_id AND a.product_id = a1.product_id
WHERE UPPER(a.status) = 'ACTIVE';
""")